In [ ]:
import os
import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
from natsort import natsorted
from IPython.display import display
from scipy.stats import bootstrap
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

In [2]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=10, 
n_informative=5, n_redundant=5, random_state=1)
print(X.shape, y.shape)

(1000, 10) (1000,)


In [ ]:
# evaluate RFE for classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
# define dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5,
    random_state=1)
# create pipeline
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5) 
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1) # report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
def get_5fold_score(X, y, p):
    '''Get 5-fold score(1 - error rate) associated with speficed number of features p
    '''
    total_score = 0 
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=11)
    for ti, vi in skf.split(X, y):
        X_train, X_val = X.iloc[ti, :], X.iloc[vi, :]
        y_train, y_val = y[ti], y[vi]
        model = LogisticRegression(C=10000, solver='liblinear', max_iter=50)
        rfe = RFE(model, n_features_to_select=p)
        rfe = rfe.fit(X_train, y_train)
        total_score += rfe.score(X_val, y_val)

    return total_score/5

def get_best_p(X, y):
    '''Get the "best" number of features p which has the largest score
    Parameters
    ----------
    X : {array-like, data frame} of shape (n_samples, n_features)
        Training vector, where `n_samples` is the number of samples and
        `n_features` is the number of features.
    y : array-like of shape (n_samples,)
        Target vector relative to X.
    
    Returns
    -------
    best_p : number of features which gets the largest score (1 - error rate)
    max_score : maximum of 5-fold average score
    '''
    best_p = 1
    max_score = 0
    for p in range(1, X.shape[1]):
        cur_score = get_5fold_score(X, y, p)
        if(cur_score > max_score):
            best_p = p
            max_score = cur_score

    return best_p, max_score

In [ ]:
import warnings
warnings.filterwarnings("ignore")

y = np.array(train_label)
bl = 1 # best number to break time series -- l
bp = 1 # best number of features -- p
max_score = 0 # largest score
l = 1
for df in TDF_df_list: 
    X = df.iloc[train_index, :]
    cur_p, cur_score = get_best_p(X, y)
    print(l)
    if cur_score > max_score:
        bl = l
        bp = cur_p
        max_score = cur_score
    l += 1